# Get-started (Community version)

**Requirements** - In order to benefit from this tutorial, you will need:
- A basic understanding of Machine Learning
- A python environment

**Learning Objectives** - By the end of this tutorial, you should be able to:
- Create and develop a new promptflow run
- Evaluate the run with a evaluation flow
- Deploy the flow to a http endpoint.

**Motivations** - This guide will walk you through the main user journey of prompt flow code-first experience. You will learn how to create and develop your first prompt flow, test and evaluate it, then deploy it to production.

## 0. Install dependent packages

In [ ]:
%pip install -r ../../requirements.txt

## 1. Create necessary connections
Connection helps securely store and manage secret keys or other sensitive credentials required for interacting with LLM and other external tools for example Azure Content Safety.

In this notebook, we will use flow `web-classification` which uses connection `azure_open_ai_connection` inside, we need to set up the connection if we haven't added it before. After created, it's stored in local db and can be used in any flow.

Prepare your Azure Open AI resource follow this [instruction](https://learn.microsoft.com/en-us/azure/cognitive-services/openai/how-to/create-resource?pivots=web-portal) and get your `api_key` if you don't have one.

In [ ]:
import json
from promptflow import PFClient
from promptflow.entities import AzureOpenAIConnection

# client can help manage your runs and connections.
pf = PFClient()

In [ ]:
try:
    conn_name = "azure_open_ai_connection"
    conn = pf.connections.get(name=conn_name)
    print("using existing connection")
except:
    # Follow https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/create-resource?pivots=web-portal to create an Azure Open AI resource.
    connection = AzureOpenAIConnection(
        name=conn_name,
        api_key="<test_key>",
        api_base="<test_base>",
        api_type="azure",
        api_version="<test_version>",
    )

    conn = pf.connections.create_or_update(connection)
    print("successfully created connection")

print(conn)

## 2. Create a new run

`web-classification` is a flow demonstrating multi-class classification with LLM. Given an url, it will classify the url into one web category with just a few shots, simple summarization and classification prompts.

### Set flow path

In [ ]:
flow = "../../flows/standard/web-classification"  # path to the flow directory

### Test flow locally

In [ ]:
# Test flow
flow_inputs = {
    "url": "https://play.google.com/store/apps/details?id=com.twitter.android",
}
flow_result = pf.test(flow=flow, inputs=flow_inputs)
print(f"Flow result: {flow_result}")

In [ ]:
# Test single node in the flow
node_name = "fetch_text_content_from_url"
node_inputs = {
    "url": "https://play.google.com/store/apps/details?id=com.twitter.android"
}
flow_result = pf.test(flow=flow, inputs=node_inputs, node=node_name)
print(f"Node result: {flow_result}")

### Create run against multi-line data


In [ ]:
data = "../../flows/standard/web-classification/data.jsonl"  # path to the data file

# create run with default variant
base_run = pf.run(flow=flow, data=data, stream=True)

In [ ]:
details = pf.get_details(base_run)
details.head(10)

## 3. Evaluate your run
Then you can use an evaluation method to evaluate your flow. The evaluation methods are also flows which use Python or LLM etc., to calculate metrics like accuracy, relevance score.

In this notebook, we use `classification-accuracy-eval` flow to evaluate. This is a flow illustrating how to evaluate the performance of a classification system. It involves comparing each prediction to the groundtruth and assigns a "Correct" or "Incorrect" grade, and aggregating the results to produce metrics such as accuracy, which reflects how good the system is at classifying the data.

### Run evaluation against run

In [ ]:
eval_flow = "../../flows/evaluation/classification-accuracy-eval"

eval_run = pf.run(
    flow=eval_flow,
    data="../../flows/standard/web-classification/data.jsonl",  # path to the data file
    run=base_run,  # specify base_run as the run you want to evaluate
    column_mapping={
        "groundtruth": "${data.answer}",
        "prediction": "${run.outputs.category}",
    },  # map the url field from the data to the url input of the flow
    stream=True,
)

In [ ]:
details = pf.get_details(eval_run)
details.head(10)

In [ ]:
metrics = pf.get_metrics(eval_run)
print(json.dumps(metrics, indent=4))

In [ ]:
pf.visualize([base_run, eval_run])

### Create another run with different variant node

In this example, `web-classification`'s node `summarize_text_content` has two variants: `variant_0` and `variant_1`. The difference between them is the inputs parameters:

variant_0:

    - inputs:
        - deployment_name: gpt-35-turbo
        - max_tokens: '128'
        - temperature: '0.2'
        - text: ${fetch_text_content_from_url.output}

variant_1:

    - inputs:
        - deployment_name: gpt-35-turbo
        - max_tokens: '256'
        - temperature: '0.3'
        - text: ${fetch_text_content_from_url.output}


You can check the whole flow definition at [flow.dag.yaml](../../flows/standard/web-classification/flow.dag.yaml)

In [ ]:
# use the variant1 of the summarize_text_content node.
variant_run = pf.run(
    flow=flow,
    data=data,
    variant="${summarize_text_content.variant_1}",  # here we specify node "summarize_text_content" to use variant 1 verison.
    stream=True,
)

In [ ]:
details = pf.get_details(variant_run)
details.head(10)

### Run evaluation run against variant run

In [ ]:
eval_flow = "../../flows/evaluation/classification-accuracy-eval"

eval_run_variant = pf.run(
    flow=eval_flow,
    data="../../flows/standard/web-classification/data.jsonl",  # path to the data file
    run=variant_run,  # use run as the variant
    column_mapping={
        "groundtruth": "${data.answer}",
        "prediction": "${run.outputs.category}",
    },  # map the url field from the data to the url input of the flow
    stream=True,
)

In [ ]:
details = pf.get_details(eval_run_variant)
details.head(10)

In [ ]:
metrics = pf.get_metrics(eval_run_variant)
print(json.dumps(metrics, indent=4))

In [ ]:
pf.visualize([eval_run, eval_run_variant])

# Next Steps

Learn more on how to:
- manage connection: [connection.ipynb](../../connections/connection.ipynb)
- manage run: [run.ipynb](../advanced-run-management/run.ipynb)
- deploy the flow as a local http endpoint: [deploy.md](../flow-deploy/deploy.md)
- create the flow as a run in azure: [quickstart-azure.ipynb](./quickstart-azure.ipynb)